In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.optimize
from scipy.interpolate import interp1d

from datetime import datetime
import pytz

import trfp

central_time = pytz.timezone('US/Central')

matplotlib.rc('xtick', labelsize=10)     
matplotlib.rc('ytick', labelsize=10)
matplotlib.rc('axes', titlesize=16)
matplotlib.rc('axes', labelsize=12)
plt.rcParams["font.family"] = "serif"
sns.set(style="darkgrid")

In [ ]:
# Trolley azimuthal uncertainty couples to dm/dphi

# calculate dm/dphi from trolley, run 3956
# interpolate the moment vs phi to standard phi values (36,000 points)

corrected_df = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3956_corrected_df')
corrected_df = corrected_df.sort_values(by=['tr_phi'])

trm = 'tr,m1'

new_phis = np.arange(36000)/100
old_phis = np.append(corrected_df['tr_phi'].values - 360,
                     np.append(corrected_df['tr_phi'].values,
                               corrected_df['tr_phi'].values + 360)
                    )
old_ms = np.append(corrected_df[trm].values,
                   np.append(corrected_df[trm].values,
                             corrected_df[trm].values)
                  )/61.79  # convert to ppm

new_ms = interp1d(old_phis, old_ms, kind='slinear')(new_phis)

dm_dphi = (np.roll(new_ms, -1) - np.roll(new_ms, 1))/(2*0.01)
dm_dz = dm_dphi * (1/124.13)  # deg/mm

rms_dm_dz = np.std(dm_dz)
print 'RMS dm/dz is ' + str(rms_dm_dz) + ' ppm/mm.'
print 'Mean dm/dz is ' + str(np.mean(dm_dz)) + ' ppm/mm.'

fig, axs = plt.subplots(3,1)
plt.sca(axs[0])
plt.plot(new_phis, new_ms)
axs[0].set_xlim((0,360))
axs[0].set_ylim((750,900))

plt.sca(axs[1])
plt.plot(new_phis, dm_dz, '.')
axs[1].set_xlim((0,360))
# axs[1].set_ylim((48000,56000))

plt.sca(axs[2])
plt.plot(old_phis, old_ms)
axs[2].set_xlim((0,360))
axs[2].set_ylim((750,900))

fig.tight_layout()
fig.set_size_inches(12,8)